In [ ]:
#include "TFile.h"
#include "TTree.h"
#include "TBrowser.h"
#include "TH2.h"
#include "TRandom.h"
#include "TCanvas.h"
#include "TMath.h"
#include "TROOT.h"
#include <string>
#include <iostream>
#include "TTreeReader.h"

//#include "../test/Event.h"

In [ ]:
//TDRStyle
gSystem->Load("style-CMSTDR.C");

In [ ]:
int CreateRandomTree(std::string name){
    /*
    Create trees for tries   
    */
    gROOT->Reset();
    /*
    Create Ttrees for tries 
    */
    std::string Name = name+".root";
    //convierto en cadena de caracteres constante (es lo que recibe la función)
    const char * constName = Name.c_str();
    //const char * constname = name.c_str();
    TFile hfile(constName,"RECREATE");
    //The Ttrees must to have the same name "tree" in this case, "Delphes"
    TTree *T = new TTree("tree","test Ttree");
    Double_t px,py,pz;
    UShort_t i;
    T->Branch("px",&px,"px/D");
    T->Branch("py",&py,"py/D");
    T->Branch("pz",&pz,"pz/D");
    T->Branch("i",&i,"i/s");
    for (i = 0; i < 65000; i++) {
        px = (Double_t) gRandom->Binomial(500,0.05);
        pz = (Double_t) gRandom->Landau(-10,100);
        py = (Double_t) gRandom->Gaus(0,1000);
        T->Fill();
    }

    //T->Print();
    hfile.Write();
    hfile.Close();
   return 0;
}

In [ ]:
void* TypeAllocate(std::string str){
    /*
    C : a character string terminated by the 0 character
    B : an 8 bit signed integer (Char_t)
    b : an 8 bit unsigned integer (UChar_t)
    S : a 16 bit signed integer (Short_t)
    s : a 16 bit unsigned integer (UShort_t)
    I : a 32 bit signed integer (Int_t)
    i : a 32 bit unsigned integer (UInt_t)
    F : a 32 bit floating point (Float_t)
    D : a 64 bit floating point (Double_t)
    L : a 64 bit signed integer (Long64_t)
    l : a 64 bit unsigned integer (ULong64_t)
    O : [the letter o, not a zero] a boolean (Bool_t)
    */ 
    if (str=="C") {
        char *ptr=new char;
        return (void*)ptr;
    }
    if (str=="B"){
        Char_t *ptr=new Char_t;
        return (void*)ptr;
    }
    if (str=="b"){
        UChar_t *ptr=new UChar_t;
        return (void*)ptr;
    }
    if (str=="S"){
        Short_t *ptr=new Short_t;
        return (void*)ptr;
    }
    if (str=="s"){
        UShort_t *ptr=new UShort_t;
        return (void*)ptr;
    }
    if (str=="I"){
        Int_t *ptr=new Int_t;
        return (void*)ptr;
    }
    if (str=="i"){
        UInt_t *ptr=new UInt_t;
        return (void*)ptr;
    }
    if (str=="F"){
        Float_t *ptr=new Float_t;
        return (void*)ptr;
    }
    if (str=="D"){
        Double_t *ptr=new Double_t;
        return (void*)ptr;
    }
    if (str=="L"){
        Long64_t *ptr=new Long64_t;
        return (void*)ptr;
    }
    if (str=="l"){
        ULong64_t *ptr=new ULong64_t;
        return (void*)ptr;
    }
    if (str=="O"){
        Bool_t *ptr=new Bool_t;
        return (void*)ptr;
    }
    return NULL;
}


In [ ]:
//CreateRandomTree("tree1");
//CreateRandomTree("tree2");

In [ ]:
TChain * chain1 = new TChain("tree");
TChain * chain2 = new TChain("tree");
chain1->Add("tree1.root");
chain1->Add("tree2.root");
chain2->Add("tree3.root");
chain2->Add("tree4.root");

In [ ]:
//chain1->Print()

In [ ]:
//tree->Show(120000)

In [ ]:
//chain1->GetEntries()

In [ ]:
//   Specify address where to read the event object
//   In the program writing the files, the event was stored in a branch called "py,px,pz, and i"
Double_t py;
Double_t px;
Double_t pz;
UShort_t i;
chain1->SetBranchAddress("px", &px);
chain1->SetBranchAddress("py", &py);
chain1->SetBranchAddress("pz", &pz);
chain1->SetBranchAddress("i", &i);

In [ ]:
//chain1->SetBranchStatus("*", 0); /*Todas las ramas out*/
//chain1->SetBranchStatus("px", 1);   

In [ ]:
Max=60.0;
Min=0.0;
NBines=20;  

In [ ]:
//   Create an histogram
TH1D *Pxst = new TH1D("Px","Histograma de Px",NBines,Min,Max);
//Pxst->SetName("Px");
//Pxst->SetTitle("Histograma de Px");
Pxst->SetFillColor(1);

TH1D *Pyst = new TH1D("Py","Histograma_de_Py",NBines,Min,Max);
//Pxst->SetName("Py");
//Pxst->SetTitle("Histograma de Py");
Pxst->SetFillColor(33);

TH1D *Pzst = new TH1D("Pz","Histograma de Pz",NBines,Min,Max);
//Pxst->SetName("Pz");
//Pxst->SetTitle("Histograma de Pz");
Pxst->SetFillColor(99);

In [ ]:
Int_t n =chain1->GetEntries();
for (Int_t j=0;j<n;j++) {
    chain1->GetEvent(j);              //read complete accepted event in memory
    Pxst->Fill(px);              //Fill histogram with 
    Pyst->Fill(py);
    Pzst->Fill(pz);    
}

In [ ]:
THStack *hs = new THStack();

In [ ]:
hs->Add(Pxst);
hs->Add(Pyst);
hs->Add(Pzst);

In [ ]:
TCanvas *cst = new TCanvas();
cst->Divide(2,2);

In [ ]:
cst->cd(1);
hs->Draw();

In [ ]:
cst->cd(2);
gPad->SetGrid();
hs->Draw("nostack,e1p");

In [ ]:
cst->cd(3);
gPad->SetFrameFillColor(17);
gPad->SetTheta(3.77);
gPad->SetPhi(2.9);
hs->Draw("lego1");

In [ ]:
gROOT->GetListOfCanvases()->Draw()

## Leer Un arbol sin saber espesificamente las ramas

In [ ]:
Bool_t Debug=1;
std::map<std::string,TChain*> ChainMap;
std::map<std::string,void*> ptrMap;
auto Array=chain1->GetListOfBranches();
Int_t ArrayEntries=Array->GetEntries();
for ( Int_t j=0 ; j<ArrayEntries ; j++ ){
    auto object = Array->At(j);
    std::string name=object->GetName();
    if(Debug){
        object->Print();
        std:cout<<"Name: "<<name<<std::endl;
    }
    //Getting the type of the branch
    std::string Title=object->GetTitle();
    std::size_t found=Title.find_last_of('/');
    if (found!=std::string::npos){
        std::string Type = Title.substr(found+1);
        //   Specify address where to read the event object
        if(Debug) std::cout<<"Type: "<<Type<<std::endl;
        //Aloco un puntero al tipo de dato del branch, y lo guardo en el map, con el key nombre
        ptrMap[name]=TypeAllocate(Type);
        if(Debug) std::cout<<"ptr: "<<ptrMap[name]<<std::endl;
        //El branch se va a leer de la dirección del puntero gurdado en el PointMap[name]
        chain1->SetBranchAddress(name.c_str(), ptrMap[name]);
    }
    else{
        std::cout<<"Error no type defined in "<<Title<<std::endl;
    }
    
    
}

In [ ]:
for ( Int_t j=0 ; j<ArrayEntries ; j++ ){
    auto object = Array->At(j);
    std::string name=object->GetName();
    if(Debug){
        object->Print();
        std:cout<<"Name: "<<name<<std::endl;
    }
    //Getting the type of the branch
    std::string Title=object->GetTitle();
    std::size_t found=Title.find_last_of('/');
    if (found!=std::string::npos){
        std::string Type = Title.substr(found+1);
        //   Specify address where to read the event object
        if(Debug) std::cout<<"Type: "<<Type<<std::endl;
        //Aloco un puntero al tipo de dato del branch, y lo guardo en el map, con el key nombre
        ptrMap[name]=TypeAllocate(Type);
        if(Debug) std::cout<<"ptr: "<<ptrMap[name]<<std::endl;
        //El branch se va a leer de la dirección del puntero gurdado en el PointMap[name]
        chain1->SetBranchAddress(name.c_str(), ptrMap[name]);
    }
    else{
        std::cout<<"Error no type defined in "<<Title<<std::endl;
    }
    
    
}

In [ ]:
ptrMap["pz"]

In [ ]:
std::map<std::string,TChain*> ChainMap;

In [ ]:
//   Specify address where to read the event object
//   In the program writing the files, the event was stored in a branch called "py,px,pz, and i"
Double_t py;
chain1->SetBranchAddress("px", &px);

In [ ]:
nmap["d1"]=d1;

In [ ]:
chain1

In [ ]:
Double_t *d1=new Double_t[3];

In [ ]:
#include<TString.h>


In [ ]:
nmap["d1"]=d1;

In [ ]:
TypeR("1")